# Static Analysis

In [7]:
import astroid
from typing import Optional, NewType, List


## Linting


In [2]:
# Before continuing, install the 'pylint' library

"""
!pip install pylint
"""

'\n!pip install pylint\n'

In [3]:
# Pylint is handy at finding some common errors in your code. 
# For example, consider a function that adds all of an author’s cookbooks to an existing  list:
# (save this code as lint_example.py)

def find_author(author_name):
    return

def add_authors_cookbooks(author_name: str, cookbooks: List[str] = []) -> bool:
    author = find_author(author_name)
    if author is None:
        assert False, "Author does not exist"
    else:
        for cookbook in author.get_cookbooks():
            cookbooks.append(cookbook)
        return True


In [4]:
# Then run pylint against the last code. It will report a series of issues with the code, as follows:

!pylint lint_example.py


************* Module lint_example
lint_example.py:1:0: C0114: Missing module docstring (missing-module-docstring)
lint_example.py:3:0: C0116: Missing function or method docstring (missing-function-docstring)
lint_example.py:3:16: W0613: Unused argument 'author_name' (unused-argument)
lint_example.py:6:0: W0102: Dangerous default value [] as argument (dangerous-default-value)
lint_example.py:6:0: C0116: Missing function or method docstring (missing-function-docstring)
lint_example.py:7:4: E1128: Assigning result of a function call, where the function returns None (assignment-from-none)

-----------------------------------
Your code has been rated at 0.00/10



### Writing Your Own Pylint Plug-in


In [9]:
# Consider the following code from Chapter 4:

class HotDog:
    pass

ReadyToServeHotDog = NewType("ReadyToServeHotDog", HotDog)
def prepare_for_serving() -> ReadyToServeHotDog:
    # snip preparation
    hotdog = HotDog()
    return ReadyToServeHotDog(hotdog)


In [10]:
# In the last snippet, we used the NewType with a 'blessed function' that enforce 
# the constraints tied to that type. But standard type checkers can't enforce it.
# But pylint can enforce it with a custom plug-in, as follows:

from pylint.checkers import BaseChecker
from pylint.interfaces import IAstroidChecker
from pylint.lint.pylinter import PyLinter


class ServableHotDogChecker(BaseChecker):
    __implements__ = IAstroidChecker
    name = 'unverified-ready-to-serve-hotdog'
    priority = -1
    msgs = {
        'W0001': (
            'ReadyToServeHotDog created outside of hotdog.prepare_for_serving.',
            'unverified-ready-to-serve-hotdog',
            'Only create a ReadyToServeHotDog through hotdog.prepare_for_serving.'
        ),
    }

    def __init__(self, linter: Optional[PyLinter] = None):
        super(ServableHotDogChecker, self).__init__(linter)
        self._is_in_prepare_for_serving = False

    def visit_functiondef(self, node: astroid.scoped_nodes.FunctionDef):
        if (
            node.name == "prepare_for_serving" and
            node.parent.name == "hotdog" and
            isinstance(node.parent, astroid.scoped_nodes.Module)
        ):
            self._is_in_prepare_for_serving = True

    def leave_functiondef(self, node: astroid.scoped_nodes.FunctionDef):
        if (
            node.name == "prepare_for_serving" and
            node.parent.name == "hotdog" and
            isinstance(node.parent, astroid.scoped_nodes.Module)
        ):
            self._is_in_prepare_for_serving = False

    def visit_call(self, node: astroid.node_classes.Call):
        if node.func.name != 'ReadyToServeHotDog':
            return
        if self._is_in_prepare_for_serving:
            return

        self.add_message('unverified-ready-to-serve-hotdog', node=node,)

    def register(linter: PyLinter):
        linter.register_checker(ServableHotDogChecker(linter))


In [13]:
# This linter checks that if someone creates a ReadyToServeHotDog, 
# it's done in a function named 'prepare_for_serving', and live in a module 'hotdog'. 
# If we create another function that returns a ReadyToServeHotDog, pylint will catch it:

def create_hot_dog() -> ReadyToServeHotDog:
    hot_dog = HotDog()
    return ReadyToServeHotDog(hot_dog)


## Other Static Analyzers


### Complexity Checkers


In [18]:
# Before continuing, install the 'mccabe' cyclomatic complexity checker

"""
!pip install mccabe
"""

'\n!pip install mccabe\n'

In [24]:
# Considet the following snippet, extracted from the mccabe library itself
# (save the file as mccabe_snippet.py):

class PathNode:
    def __init__(self, string, look) -> None:
        pass

class PathGraphingAstVisitor:
    def __init__(self) -> None:
        pass
    
    def _subgraph_parse(self, node, pathnode, extra_blocks):
        """parse the body and any `else` block of `if` and `for` statements"""
        loose_ends = []
        self.tail = pathnode
        self.dispatch_list(node.body)
        loose_ends.append(self.tail)
        for extra in extra_blocks:
            self.tail = pathnode
            self.dispatch_list(extra.body)
            loose_ends.append(self.tail)
        if node.orelse:
            self.tail = pathnode
            self.dispatch_list(node.orelse)
            loose_ends.append(self.tail)
        else:
            loose_ends.append(pathnode)
        if pathnode:
            bottom = PathNode("", look='point')
        for le in loose_ends:
            self.graph.connect(le, bottom)
        self.tail = bottom

In [25]:
# Then we can run mccabe to find functions that has a cyclomatic complexity greater than a threshold
# (5 in this case), as follows:

!python -m mccabe --min 5 mccabe_snippet.py

11:4: 'PathGraphingAstVisitor._subgraph_parse' 5


As an heuristic, higher McCabe complexity numbers mean higher code complexity, which can make it harder to maintain.

### Custom complexity checkers

We can use whitespace checking as another complexity measure: how many levels of indentation there are in a single Python file.

In [30]:
# A simple implementation of the last concept:

def get_amount_of_preceding_whitespace(line: str) -> int:
    """Replace tabs with 4 spaces"""
    tab_normalized_text = line.replace("\t", " ")
    return len(tab_normalized_text) - len(tab_normalized_text.lstrip())

def get_average_whitespace(filename: str) -> None:
    with open(filename) as file_to_check:
        whitespace_count = [
            get_amount_of_preceding_whitespace(line)
            for line in file_to_check
            if line != ""
        ]
        average = sum(whitespace_count) / len(whitespace_count) / 4
        print(f"Avg indentation level for {filename}: {average}")
